In [2]:
import os.path as op
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import sys
base = "/projects/master-theses/henschell"
sys.path.append(base)
sys.path.append(op.join(base, "SuperResSurfer"))

from collections import defaultdict
from FastInfantSurfer.data_util.data_utils import filter_blank_slices_thick, transform_axial, transform_sagittal

In [3]:
# Read list of validation subjects
datadir = "/projects/datasets/dHCP/Data/"

csv = "FastInfantSurfer/data/dataset_split_large_validation_t1t2_meta_slices.tsv"
valset = pd.read_csv(op.join(base, csv), sep="\t")

In [4]:
valset.head()

,Path,SubjectFix,scan_age,sex,Resolution,NumberSlices,CumulativeNumberSlices,NumberSlices_coronal,CumulativeNumberSlices_coronal,NumberSlices_axial,CumulativeNumberSlices_axial,NumberSlices_sagittal,CumulativeNumberSlices_sagittal
0,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00694XX19_ses-201800,27.29,female,0.5,153,153,153,153,131,131,153,153
1,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00666XX15_ses-198200,30.00,female,0.8,97,250,97,250,90,221,97,250
2,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00526XX15_ses-150500,32.43,female,1.0,82,332,82,332,79,300,82,332
3,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00305XX08_ses-98101,32.71,female,0.5,178,510,178,510,153,453,178,510
4,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC01153AN07_ses-102030,33.71,female,0.8,120,630,120,630,101,554,120,630


In [23]:
num_sl["axial"] == num_sl["sagittal"]
op.join(base, csv[:-4] + ".tsv")

'/groups/ag-reuter/projects/master-theses/henschell/FastInfantSurfer/data/dataset_split_large_validation_t1t2_meta_slices.tsv'

In [11]:
valset["Path"] = datadir + valset["SubjectFix"]
sdirs = valset["Path"]

In [19]:
from FastInfantSurfer.data_util.data_utils import read_classes_from_lut
sys.path.append("/projects/DeepSurfer/FastSurfer")
sys.path.append("/projects/DeepSurfer/FastSurfer/FastSurfer")
import FastSurfer.FastSurferCNN.data_loader.data_utils as du

selfmapping="none"
selflut=read_classes_from_lut("/projects/master-theses/henschell/FastInfantSurfer/configs/FastInfantSurfer_dHCP_full_LUT.tsv")
selflabels, selflabels_sag= du.get_labels_from_lut(selflut, ("Left-", "ctx-rh"))
selflateralization=du.unify_lateralized_labels(selflut, ["Left-", "Right-"])

In [25]:
LookUpRes = {0.5: "_min", 0.8: "_08", 1.0: "", 0.7: ""}
preproc_hook = {"sagittal": transform_sagittal, "axial": transform_axial}

ln = "_desc-drawem88_dseg" # "_desc-drawem88_dseg"
lf = ".nii.gz"
num_sl = {"coronal": [], "axial": [], "sagittal": []}
for sb, res in zip(valset["SubjectFix"], valset["Resolution"]):
    aname = sb + ln + LookUpRes[res] + lf
    labels = np.asanyarray(nib.load(op.join(datadir, sb, aname)).dataobj, dtype=np.int16)
    mapped_aseg, mapped_aseg_sag = du.map_aparc_aseg2label(labels, aseg_nocc=None,
                                                           processing=selfmapping,
                                                           labels=selflabels, labels_sag=selflabels_sag,
                                                           sagittal_lut_dict=selflateralization
                                                            )
    for plane in ["coronal", "axial", "sagittal"]:
        if plane == "sagittal":
            labels=mapped_aseg_sag
        else:
            labels=mapped_aseg
        labels = preproc_hook.get(plane, lambda x: x)(labels)
        vols = filter_blank_slices_thick({"Labels": labels}, threshold=2)
        num_sl[plane].append(vols["Labels"].shape[2])
    

In [24]:
csv

'FastInfantSurfer/data/dataset_split_large_validation_t1t2_meta_slices.tsv'

In [27]:
for plane in ["axial", "sagittal", "coronal"]:
    print(np.sum(num_sl[plane]))
    valset["NumberSlices_" + plane] = num_sl[plane]
    valset["CumulativeNumberSlices_" + plane] = np.cumsum(num_sl[plane])

11809
11344
13916


In [23]:
np.sum(valset["NumberSlices_sagittal"])

11344

In [26]:
valset.to_csv(op.join(base, csv[:-4] + "_corrected.tsv"), sep="\t", index=False)

In [76]:
from SuperResSurfer.SuperResSegm.data_loader import loader, data_utils 
from SuperResSurfer.SuperResSegm import train
from SuperResSurfer.SuperResSegm.utils import metrics
from SuperResSurfer.SuperResSegm.utils.load_config import load_config
from SuperResSurfer.SuperResSegm.utils.misc import plot_predictions
import FastInfantSurfer.data_util.data_utils as du

In [202]:
cfg = load_config(f"{base}/FastInfantSurfer/VINNA/config/FastSurferDDB2_Infant_bigMix_T2_AdamW_Cos_3x3F_71_1.0BR_coronal/config.yaml")
cfg.DATA.PATH_HDF5_VAL = f'{base}/FastInfantSurfer/hdf5_sets/validation_bigMix_dHCP_coronal.hdf5'
cfg.TRAIN.BATCH_SIZE = 8
val_loader = loader.get_dataloader(cfg, "val")
slicepersubject = valset["NumberSlices"]
subject = valset["SubjectFix"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lut = du.read_classes_from_lut(op.join(base, "FastInfantSurfer/configs/FastInfantSurfer_dHCP_LUT.tsv"))
label_list = lut["MappedID"][1:]
torch_labels = torch.from_numpy(lut["MappedID"].values)

# Set up metric arrays
metric_store = defaultdict(lambda: defaultdict(lambda: defaultdict))
i = 0
start_index = 0

pred_vol = torch.zeros((256, 256, slicepersubject[0]), dtype=torch.long).to(device)
label_vol = torch.zeros((256, 256, slicepersubject[0]), dtype=torch.long) # Label can be on cpu (avoid copying of data)
start = True
if torch.max(torch.unique(pred_vol)) > 23:
    print("Pred_vol", torch.unique(pred_vol))
    plot_predictions(batch['image'], batch['label'], batch['label'], "SVINN Tests", None, None, "val")
    

13604


In [204]:
# Test batch assignment

for curr_iter, batch in enumerate(val_loader):
    # Load images and labels
    images, labels= batch['image'].to(device), batch['label']
    
    # predicition theoretically ... 
    pred = labels.to(device)
    
    #if start == True:
    #plot_predictions(batch['image'], batch['label'], batch['label'], "SVINN Tests", None, None, "val")

    
    # Assemble batches into volume
    pred = pred.permute(1, 2, 0) #torch.max(pred, dim=1).permute(1, 2, 0)    
    shape_left = pred_vol[:, :, start_index:start_index + pred.shape[2]].shape[2]  
    pred_vol[:, :, start_index:start_index + pred.shape[2]] = pred[..., :shape_left]

    # Assemble label into volume
    labels = labels.permute(1, 2, 0)  
    label_vol[:, :, start_index:start_index + pred.shape[2]] = labels[..., :shape_left]
    assert torch.equal(pred_vol.cpu(), label_vol)
    start_index += pred.shape[2]

    if start_index >= slicepersubject[i]:
        # Get subject name and increase cumsize by one (=next subject)
        sid = subject[i]
        sf = valset["Resolution"][i]
        i += 1
        start_index = 0 + pred.shape[2] - shape_left
        
        
        # Image finished, calculate and store results, reset pred_prob 
        metric_store[sf][sid] = metrics.evaluate_metrics(label_vol, pred_vol.cpu(), label_list)
        pred_vol = torch.zeros((256, 256, slicepersubject[i]), dtype=torch.long).to(device)
        label_vol = torch.zeros((256, 256, slicepersubject[i]), dtype=torch.long) # Label can be on cpu (avoid copying of data)

        # Add overlapping info (if mixed batch)
        pred_vol[:, :, start_index:start_index + pred[..., shape_left:].shape[2]] += pred[..., shape_left:]
        label_vol[:, :, start_index:start_index + pred[..., shape_left:].shape[2]] += labels[..., shape_left:]
        

KeyError: 90

In [194]:
b = metrics.evaluate_metrics(label_vol, pred_vol, label_list)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [195]:
pred_vol.shape

torch.Size([256, 256, 153])

In [196]:
import SimpleITK as sitk
def dsc(gt, pred, label_list):
    sitk.ProcessObject.SetGlobalDefaultCoordinateTolerance(1e-04)
    dsc = []
    for k in label_list:
        if np.unique(gt[gt == k]).size == 0:
            print(f"Attention! Label {k} not found in ground truth")
            dsc.append(np.nan)
        else:
            # threshold the image and load as SITK image input image
            segmentation = sitk.GetImageFromArray(np.where(pred == k, 1, 0))
            gold_standard = sitk.GetImageFromArray(np.where(gt == k, 1, 0))

            # Compute overlap
            ruler = sitk.LabelOverlapMeasuresImageFilter()
            try:
                ruler.Execute(segmentation, gold_standard)
                # Append results
                dsc.append(ruler.GetDiceCoefficient())
            except RuntimeError:
                print(f"Attention! Label {k} generated RunTimeError")
                dsc.append(np.nan)
    return dsc

def dsc_numpy(gt, pred, label_list):
    intersect = np.zeros(len(label_list))
    union = np.zeros(len(label_list))

    for i in range(len(label_list)):
        label = label_list[i]
        set_pred = (pred == label).float().ge(1).sum().item()
        set_gt = (gt == label).float().ge(1).sum().item()
        intersect[i] = ((pred == label).float() + (gt == label).float()).eq(2).sum().item()
        union[i] = ((pred == label).float() + (gt == label).float()).ge(1).sum().item()
        print(f"Label: {label}, pred total: {set_pred}, gt_total: {set_gt}")
    print(intersect, union)
    return 2 * intersect / union

In [201]:
dsc(label_vol, pred_vol.cpu(), label_list.values)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [198]:
np.unique(label_vol)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [199]:
metric_store = defaultdict(lambda: defaultdict(lambda: defaultdict))

In [200]:
metric_store[sf][sid] = b

In [205]:
metric_store

defaultdict(<function __main__.<lambda>()>,
            {0.8: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'sub-CC00666XX15_ses-198200': {'vs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                                  0., 0., 0., 0., 0., 0.]),
                           'dice': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                  1., 1., 1., 1., 1., 1.]),
                           'surfaceAverageHausdorff': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                                  0., 0., 0., 0., 0., 0.])},
                          'sub-CC01153AN07_ses-102030': {'vs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                                  0., 0., 0., 0., 0., 0.]),
                           'dice': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          

In [46]:
import h5py
hf = h5py.File(cfg.DATA.PATH_HDF5_VAL, "r")

In [47]:
hf.keys() 

<KeysViewHDF5 ['256']>

In [48]:
hf['256'].keys()

<KeysViewHDF5 ['aseg_dataset', 'orig_dataset', 'subject', 't2_dataset', 'weight_dataset', 'zoom_dataset']>

In [59]:
subjects = np.asarray(hf['256']["subject"], dtype=str)

In [60]:
subjects

array(['sub-CC00694XX19_ses-201800', 'sub-CC00666XX15_ses-198200',
       'sub-CC00526XX15_ses-150500', 'sub-CC00305XX08_ses-98101',
       'sub-CC01153AN07_ses-102030', 'sub-CC00628XX18_ses-181800',
       'sub-CC01129XX16_ses-101430', 'sub-CC00946XX23_ses-15230',
       'sub-CC01218XX14_ses-147430', 'sub-CC00986AN22_ses-26730',
       'sub-CC00648XX22_ses-191100', 'sub-CC00517XX14_ses-145000',
       'sub-CC00688XX21_ses-199500', 'sub-CC00434AN14_ses-127700',
       'sub-CC00597XX21_ses-190200', 'sub-CC00313XX08_ses-100000',
       'sub-CC00550XX06_ses-157800', 'sub-CC00301XX04_ses-113001',
       'sub-CC00060XX03_ses-12501', 'sub-CC00071XX06_ses-27000',
       'sub-CC00720XX11_ses-211101', 'sub-CC00504XX09_ses-146800',
       'sub-CC00787XX21_ses-19010', 'sub-CC00377XX15_ses-119800',
       'sub-CC00068XX11_ses-20701', 'sub-CC00088XX15_ses-31801',
       'sub-CC00716XX15_ses-222800', 'sub-CC00410XX06_ses-125800',
       'sub-CC00876XX19_ses-17931', 'sub-CC00458XX13_ses-138900',
    

In [193]:
    
    if torch.max(torch.unique(pred_vol)) > 23 or torch.max(torch.unique(pred)) > 23:
        print(curr_iter)
        print("label", torch.unique(labels), labels.shape)
        print("Pred", torch.unique(pred), batch["label"].shape)
        print("Pred_vol3", torch.unique(pred_vol), pred_vol.shape)
        curr_slices = pred_vol[:, :, start_index:start_index + pred.shape[2]].permute(2, 0, 1)
        print(curr_slices.shape)
        plot_predictions(batch['image'], pred.permute(2, 0, 1), curr_slices, "SVINN Tests", None, None, "val")
        break

SyntaxError: 'break' outside loop (3253457052.py, line 9)

In [61]:
subjects == valset["SubjectFix"].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])